In [2]:
import pandas as pd
import numpy as np
import pyodbc
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import gensim

In [ ]:
host = r''
dbSQL = ''
user = ''
pwd = ''
MC0 = '''ABS AH- ALM BC- BL- CA- CAE CHG COM CU- FP- FW- GA- GBL GPP GUE HVC LBU LGR LIT MWT PLT PND PRD PWS RS- SCR SD- SDR SPP STB SW- TIC TRF UDT WST'''.split(' ')
MC1 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- BCW SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
MC2 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- BCW SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
MC3 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- SCR SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
#Consider adding MC00X in fron to all above in code os I don't need to do it later in teh code :)
#I will do this in an algorithm later by pulling in MaximoDW.dbo.Locations and grabbing DISTINCT SUBSTRING(LOCATION, 4, 8) but right now i'm lazy
where = ''
for l in MC0: where += f'OR LOCATION LIKE \'MC000{l}%\' '
for l in MC1: where += f'OR LOCATION LIKE \'MC001{l}%\' '
for l in MC2: where += f'OR LOCATION LIKE \'MC002{l}%\' '
for l in MC3: where += f'OR LOCATION LIKE \'MC003{l}%\' '
where = where[3:]
maximoSelect = f"""SELECT [

"""
woSelect= f"""SELECT [WONUM]
      ,[LOCATION]
      ,[DESCRIPTION]
  FROM [Generation].[agent].[MaximoWorkOrders]
  Where LOC = 'MC'
  and cast(STATUSDATE as date) > '01/01/2020'
  And ({where})"""
#getting data and then relableing maximo location to higher level equipment name
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+host+';DATABASE='+dbSQL+';UID='+user+';PWD='+ pwd)
WOdata = pd.read_sql(woSelect, cnxn)
for key in MC0: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC000' + key)] = 'MC000' + MC0[MC0.index(key)]
for key in MC1: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC001' + key)] = 'MC001' + MC1[MC1.index(key)]
for key in MC2: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC002' + key)] = 'MC002' + MC2[MC2.index(key)]
for key in MC3: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC003' + key)] = 'MC003' + MC3[MC3.index(key)]
WOdata = WOdata.drop(columns=['WONUM'])
WOdata

In [644]:
##Lets try the Easy Way First
y = WOdata['LOCATION']
X = WOdata['DESCRIPTION']
#Spliting Data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=53)
#processing stop_words
count_vectorizer = CountVectorizer(stop_words = 'english')
#training "fit of curve" sort of deal
count_train = count_vectorizer.fit_transform(X_train.values)
#transforms our test data using the above fit.
count_test = count_vectorizer.transform(X_test.values)
print(count_vectorizer.get_feature_names()[1000:1002])
#Classifier for words
nb_classifier = MultinomialNB()
nb_classifier.fit(count_train,y_train)
pred = nb_classifier.predict(count_test)
print(metrics.accuracy_score(y_test,pred))
cm = metrics.confusion_matrix(y_test,pred,labels = list(maximoLocations))
cm

/home/spark/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['abs', 'absent']
0.7129230551826468


ValueError: At least one label specified must be in y_true

In [ ]:
#Testing outputs for various text processors
print(WOdata.DESCRIPTION[2])
text_tokens = word_tokenize(WOdata.DESCRIPTION[2])
#text_tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(WOdata.DESCRIPTION[16])
print(text_tokens)
text_tokens = [word for word in text_tokens if not word in stop_words]
print(text_tokens)
text_tokens = [ps.stem(word) for word in text_tokens] #Stem Words
print(text_tokens)
#May need to keep numbers later on;
#text_tokens=[word for word in text_tokens if word.isalpha()]
#print(text_tokens)

In [529]:
WOdata['Edited1'] = [list() for x in range(len(WOdata.index))]
WOdata['Edited2'] = [list() for x in range(len(WOdata.index))]

def processText(text):
    text_tokens = word_tokenize(text)
    #text_tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text_tokens = [word for word in text_tokens if not word in stop_words]  ##Remove Stop Words
    text_tokens = [ps.stem(word) for word in text_tokens] #Stem Words
    text_tokens = [word for word in text_tokens if len(word) > 2] #Removing noise
    text_tokens = [word for word in text_tokens if word.isalpha()] #Remove Punctuation
    text_tokens = [ele for ele in text_tokens if ele.strip()] #remove empty spaces in list
    return text_tokens
#Applying Text Processing
WOdata['Edited1'] = WOdata['DESCRIPTION'].apply(processText)
#Concating all the words in desc
WOdata['Edited1Concat'] = WOdata['Edited1'].apply(lambda x: ' '.join(x))
#WOdata.head
#Concating all the strings into 1
allWords = ' '.join(WOdata["Edited1Concat"])
allWords = word_tokenize(allWords)

In [ ]:
##POSSIBLY DROPPING WORDS BASED ON FREQUENCY OF USE
allWordsDF = FreqDist(allWords)
allWordsDF = pd.DataFrame(list(allWordsDF.items()), columns = ["Word","Frequency"])
allWordsDF = allWordsDF.sort_values(by=['Frequency'],ascending = False)
#allWordsDF.hist(column=['Frequency'])
#allWordsDF.hist(column=['Frequency'], range= (10,100))

dictionary = allWordsDF.copy()
dictionary = dictionary[dictionary['Frequency'] < 400]
dictionary = dictionary[dictionary['Frequency'] > 3]
#dictionary.hist(column=['Frequency'])
dictionary = dictionary.reset_index(drop=True)
selectedWords = dictionary.Word.tolist()


for row in WOdata.index:
    text_tokens = WOdata.Edited1[row]
    text_tokens = [word for word in text_tokens if word in selectedWords]  ##Remove Stop Words
    WOdata.Edited2[row]=text_tokens
WOdata.Edited2

In [ ]:
##Test/Train Split
y = WOdata['LOCATION'].values
X = WOdata['Edited2'].apply(lambda x: ' '.join(x)).values

X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 21,stratify = y)
count_vectorizer = CountVectorizer()
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)
print(count_vectorizer.get_feature_names()[:10])
nb_classifier = MultinomialNB()
nb_classifier.fit(count_train,y_train)
pred = nb_classifier.predict(count_test)
print(metrics.accuracy_score(y_test,pred))
cm = metrics.confusion_matrix(y_test,pred,labels = (maximoLocations))
cm

KeyError: 'Edited2'

In [639]:
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
##Create and Test Model
#https://iq.opengenus.org/text-classification-using-k-nearest-neighbors/
#
#Testing kneighbors vs nb
y = WOdata['LOCATION'].values
X = WOdata['DESCRIPTION'].values
#X = WOdata['Edited2'].apply(lambda x: ' '.join(x)).values
#NOTES
#KNN = 1 Seems to be best over KNN = 5 which is default
#When using the post processed data, lose accuracy on KNN by .1-.25., and NB by .02-.03
#
knnAverage = 0
clsAverage = 0
bgAverage = 0
nbAverage = 0
nnAverage = 0
runs = 1
#run 1 run just to retrain model
#just doing states 0-9, you can do whatever math to change those to be weirder.
for run in range(runs):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=run*6)
    
    #generic
    count_vectorizer = CountVectorizer(stop_words = 'english')
    #Tfidf is counting but also consider importance of wors and rare wrods.
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_transformer = TfidfTransformer()
    #Training transofrmation
    count_train = count_vectorizer.fit_transform(X_train)
    X_count_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_train_tfidf = tfidf_transformer.fit_transform(count_train)
    #testing transformation
    X_new_counts = count_vectorizer.transform(X_test) #Test -> Counts
    X_test_tfidf = tfidf_vectorizer.fit_transform(X_test)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts) #counts -> freq
    
    #knn
    #https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
    knn=KNeighborsClassifier(n_neighbors = 5).fit(X_train_tfidf, y_train)
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', knn),
    ])
    text_clf.fit(X_train, y_train)
    predicted = text_clf.predict(X_test)
    print(f'RUN {run}: ')
    print(f'KNN: ',np.mean(predicted == y_test)*100)
    knnAverage += np.mean(predicted == y_test)

    
    #cls
    #https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
    cls = DecisionTreeClassifier().fit(X_train_tfidf, y_train)
    predicted = cls.predict(X_new_counts)
    print(f'CLS: ',np.mean(predicted == y_test)*100)
    clsAverage += np.mean(predicted == y_test)
    
    #bg 
    #https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
    bg = BaggingClassifier().fit(X_train_tfidf, y_train)
    predicted = bg.predict(X_new_counts)
    print(f'BG: ',np.mean(predicted == y_test)*100)
    bgAverage += np.mean(predicted == y_test)
    
    #nb
    #https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html
    nb_classifier = MultinomialNB().fit(X_train_tfidf,y_train)
    count_test = count_vectorizer.transform(X_test)
    pred = nb_classifier.predict(count_test)
    print('nb: ' + str(metrics.accuracy_score(y_test,pred) * 100))
    nbAverage += metrics.accuracy_score(y_test,pred)
    '''
    nn = MLPClassifier().fit(count_train, y_train)
    predicted = nn.predict(X_new_counts)
    print(f'NN: ',np.mean(predicted == y_test)*100)
    nnAverage += np.mean(predicted == y_test)
    '''
print("\nknn avg: " + str(knnAverage/runs))
print("cls avg: " + str(clsAverage/runs))
print("bg avg: " + str(bgAverage/runs))
print("nb avg: " + str(nbAverage/runs))
print("nn avg: " + str(nnAverage/runs))


RUN 0: 
KNN:  73.53695500116577
CLS:  51.81860573560271
BG:  59.955700629517374
nb: 65.9944042900443

knn avg: 0.7353695500116577
cls avg: 0.5181860573560271
bg avg: 0.5995570062951737
nb avg: 0.659944042900443
nn avg: 0.0


In [ ]:
#Drwawings test 
hostVM = r''
userVM = r''
pwdVM = ''
dbSQL = ''
sqlStatement=  """SELECT TOP (25) DrawingID, SUBSTRING(Name,9,100) as Name, DrawingTextAll
    FROM [Generation].[quest].[Drawings]
    WHERE DrawingTextAll IS NOT NULL AND Name LIKE '%Mill Creek%' AND DRAWINGID < \'3437674\' ORDER BY DrawingID DESC
    """
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+hostVM+';DATABASE='+dbSQL+';UID='+userVM+';PWD='+ pwdVM)
drawings = pd.read_sql(sqlStatement, cnxn)
drawings

In [ ]:
def processText(text):
    text_tokens = word_tokenize(text)
    #text_tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text_tokens = [word for word in text_tokens if not word in stop_words]  ##Remove Stop Words
    text_tokens = [ps.stem(word) for word in text_tokens] #Stem Words
    text_tokens = [word for word in text_tokens if len(word) > 2] #Removing noise
    text_tokens = [word for word in text_tokens if word.isalpha()] #Remove Punctuation
    text_tokens = [ele for ele in text_tokens if ele.strip()] #remove empty spaces in list
    return text_tokens
drawings['cleanText'] = drawings['DrawingTextAll'].apply(lambda x: (processText(str(x))))
#drawings['textCount'] = drawings['DrawingTextAll'].apply(lambda x: (count_vectorizer.transform([x])))
drawings['textCount'] = drawings['cleanText'].apply(lambda x: (count_vectorizer.transform([ ' '.join(x)])))
drawings['predictNB'] = drawings['textCount'].apply(lambda x: nb_classifier.predict( x ))
drawings['predictCLS'] = drawings['textCount'].apply(lambda x: cls.predict( x ))
drawings['predictBG'] = drawings['textCount'].apply(lambda x: bg.predict( x ))
drawings['predictKN'] = drawings['DrawingTextAll'].apply(lambda x: text_clf.predict( [x] ))
drawings = drawings.drop(columns = ['cleanText', 'textCount'])
drawings